In [ ]:
from autogen import ConversableAgent, LLMConfig, run
from autogen.agentchat.groupchat.groupchat import GroupChatManager

# todo: rename autogen.messages to autogen.events
# rename messages to events
# keep all names in autogen.message with deprecation message and import from autogen.events
from autogen.events.agent_events import InputRequestEvent
from autogen.io.event_processors import ConsoleEventProcessor
from autogen.tools import tool

llm_config = LLMConfig.from_json(path="OAI_CONFIG_LIST").where(model=["gpt-4o-mini"], api_type="openai")

llm_config

# Single agent run

In [ ]:
@tool()
def get_meaning_of_life():
    return "The meaning of life is 42."


with llm_config:
    agent = ConversableAgent(name="Alice")

get_meaning_of_life.register_for_llm(agent)

response = run(
    agent,
    message="What is the meaning of life?",
    # I renamed initial_message to message
    max_turns=10,
)


for event in response.events:
    print(f"Got event: {event}")
    if isinstance(event, InputRequestEvent):
        event.content.respond(input(event.content.prompt))

print(response.summary)

What if we want the initial message to be sent from `agent`?

In [ ]:
response = run(
    agent,
    message="What is the meaning of life?",
    sender=agent,
    # if omitted, the default sender will be a auto-created user proxy agent
    max_turns=10,
)

## With console processor

In [ ]:
@tool()
def get_meaning_of_life():
    return "The meaning of life is 42."


with llm_config:
    agent = ConversableAgent(name="Alice")

get_meaning_of_life.register_for_llm(agent)

response = run(
    agent,
    message="What is the meaning of life?",
    max_turns=5,
    user_input=True,
)

processor = ConsoleEventProcessor()
processor.process(response)

print(response.summary)

# Two agent run

In [ ]:
with llm_config:
    agent_1 = ConversableAgent(name="Jane")
    agent_2 = ConversableAgent(name="Bob")

response = run(
    sender=agent_1,
    recipient=agent_2,
    message="have a joke off",
    max_turns=5
)
# I used kwargs instead, because the original code
# response = run(agent_1, agent_2, message=...)
# is supposed to auto-create a user_proxy and send the message from it

for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.content.respond(input(event.prompt))

print(response.summary)

## With event processor

In [ ]:
with llm_config:
    agent_1 = ConversableAgent(name="Jane")
    agent_2 = ConversableAgent(name="Bob")

response = run(
    sender=agent_1,
    recipient=agent_2,
    message="have a joke off",
    max_turns=5
)

ConsoleEventProcessor().process(response)

print(response.summary)

# Multi agent run with chat manager

In [ ]:
@tool()
def submit_plan(plan: str) -> str:
    return f"Plan submitted: {plan}"


with llm_config:
    planner = ConversableAgent(
        name="Planner",
        system_message="You are a planner. Collaborate with teacher and reviewer to create lesson plans.",
    )

    reviewer = ConversableAgent(
        name="Reviewer",
        system_message="You are a reviewer. Review lesson plans against 4th grade curriculum. Provide max 3 changes.",
    )

    teacher = ConversableAgent(
        name="Teacher",
        system_message="You are a teacher. Choose topics and work with planner and reviewer. Say DONE! when finished.",
    )

    chat_manager = GroupChatManager()

submit_plan.register_for_llm(teacher)

response = run(
    planner,
    reviewer,
    teacher,
    message="Create lesson plans for 4th grade.",
    chat_manager=chat_manager,
    # a user_proxy is auto-created and then sends the message
)

for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.content.respond(input(event.prompt))

print(response.summary)

## With event processor

In [ ]:
@tool()
def submit_plan(plan: str) -> str:
    return f"Plan submitted: {plan}"


with llm_config:
    planner = ConversableAgent(
        name="Planner",
        system_message="You are a planner. Collaborate with teacher and reviewer to create lesson plans.",
    )

    reviewer = ConversableAgent(
        name="Reviewer",
        system_message="You are a reviewer. Review lesson plans against 4th grade curriculum. Provide max 3 changes.",
    )

    teacher = ConversableAgent(
        name="Teacher",
        system_message="You are a teacher. Choose topics and work with planner and reviewer. Say DONE! when finished.",
    )

    chat_manager = GroupChatManager()

submit_plan.register_for_llm(teacher)

response = run(
    planner,
    reviewer,
    teacher,
    initial_message="Create lesson plans for 4th grade.",
    chat_manager=chat_manager,
)

ConsoleEventProcessor().process(response)

print(response.summary)

# Swarm chat manager

In [ ]:
from autogen import ConversableAgent

with llm_config:
    # 1. Create our agents
    planner_message = """You are a classroom lesson planner.
    Given a topic, write a lesson plan for a fourth grade class.
    If you are given revision feedback, update your lesson plan and record it.
    Use the following format:
    <title>Lesson plan title</title>
    <learning_objectives>Key learning objectives</learning_objectives>
    <script>How to introduce the topic to the kids</script>
    """

    planner = ConversableAgent(name="planner_agent", system_message=planner_message)

    reviewer_message = """You are a classroom lesson reviewer.
    You compare the lesson plan to the fourth grade curriculum
    and provide a maximum of 3 recommended changes for each review.
    Make sure you provide recommendations each time the plan is updated.
    """

    reviewer = ConversableAgent(name="reviewer_agent", system_message=reviewer_message)

    teacher_message = """You are a classroom teacher.
    You decide topics for lessons and work with a lesson planner.
    and reviewer to create and finalise lesson plans.
    """

    teacher = ConversableAgent(name="teacher_agent", system_message=teacher_message)

    # We will unify GroupChatManager and SwarmManager and only keep the former
    chat_manager = GroupChatManager()

response = run(
    planner,
    reviewer,
    teacher,
    message="Today, let's introduce our kids to the solar system.",
    chat_manager=chat_manager,
    max_turns=5,
)

for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.content.respond(input(event.prompt))

print(response.summary)

Variation 1: no user agent

In [ ]:
with llm_config:
    chat_manager = GroupChatManager(user_agent=None)

Variation 2: user agent is manually provided instead of auto-created

In [ ]:
from autogen import UserProxyAgent

user_proxy = UserProxyAgent()

with llm_config:
    chat_manager = GroupChatManager(user_agent=user_proxy)

Variation 3: no auto-execution of function tools

In [ ]:
with llm_config:
    chat_manager = GroupChatManager(auto_execute_func_tool=False)
    # If any agent has function-based tools
    # The user needs to provide some agent able to execute the tool

# Sequential chat

Follow the design of `initiate_chats` to allow `run` to take a list of chats

# Nested Chat

Nested inside an agent. No need to change the high-level API. Need to make sure
the return type of `run` is aware that the chat may be paused in a nested chat
and is able to resume from there.